# ETL Project
## Irvine Company Apartments


In [3]:
# set environment
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Tolga\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


## Build URL for scraping cost-of-living index
- Near-future release: Use drop down for chosing region and date

In [5]:
# Base url for cost of living index by city from regional rankings
base_url = "https://www.numbeo.com/cost-of-living/region_rankings.jsp"

# Choose region
region_idx = 21   # 21 is the North America region
# Date
year = 2021       # Year of the acquisition
mid_year = False  # Not sure what this means, but there's an option to choose

In [6]:
# Date string
date_str = "%d" % (year)
if mid_year:
    date_str += "-mid"

# Region string
region_str = "%03d" % (region_idx)

# build URL for scraping 
cost_of_living_idx_url = base_url + f"?title={date_str}&region={region_str}"

In [7]:
browser.visit(cost_of_living_idx_url)
html = browser.html
# soup = BeautifulSoup(html, "html.parser")


In [55]:
rankings_df = pd.read_html(html)[1]
rankings_df.set_index("Rank", inplace=True)
# rankings_df.to_csv("data/cost_of_living.csv")
address_df = rankings_df["City"].str.split(", ", expand=True)
rankings_df["City"] = address_df[0]
rankings_df["State"] = address_df[1]

# Keep only the cities inside United States
rankings_df = rankings_df[address_df[2] == "United States"]
rankings_df = rankings_df.rename(columns = {"City": "city_name"})





### Choose cities
- Somewhat near-future release: Choose the cities from the list by searching inside df

In [57]:
cities_df = pd.read_csv("data/cities.csv")
cities = cities_df["city_name"]
pd.concat(cities_df, rankings_df)
# cities = ["Irvine, CA, United States", 
#           "San Diego, CA, United States", 
#           "Los Angeles, CA, United States", 
#           "San Jose, CA, United States"]

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [35]:
rankings_df.set_index("City").loc[cities]

,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
City,,,,,,
"Irvine, CA, United States",79.21,69.90,74.76,72.42,79.58,126.46
"San Diego, CA, United States",80.22,67.85,74.31,80.28,78.70,112.72
"Los Angeles, CA, United States",81.92,74.12,78.19,79.26,89.11,98.51
"San Jose, CA, United States",78.55,82.92,80.64,74.99,78.75,131.28
